In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

import json
import requests

In [2]:
API_KEY = "4ace0c77025132b5ae59c770af4ba01b80444161"
BASE_URL = "https://comicvine.gamespot.com/api/"

In [14]:
API_KEY = "f7ca7f049ce4f06afd6c5c1e082061cd34779ff9"
BASE_URL = "https://comicvine.gamespot.com/api/"

In [3]:
def fetch_volume_id(volume_name):
    API_URL = BASE_URL + 'volumes/'
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'filter': volume_name
    }
    try:
        response = requests.get(API_URL, headers=headers, params=params)
        response.raise_for_status()
        volumes = response.json().get('results', [])
        for volume in volumes:
            if volume_name.lower() in volume['name'].lower():
                return volume['id']
        return None
    except requests.exceptions.RequestException as e:
        print(f'Error fetching volumes: {e}')
        print(f'Response content: {response.content}')  # Print response content for debugging
        return None

In [4]:
# Function to fetch issues for a specific volume
def fetch_volume_issues(volume_id, offset=0):
    API_URL = BASE_URL + 'volume/4050-' + str(volume_id) + '/'
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'offset': offset,
        'limit': 100  # Adjust as needed, default ComicVine API limit per request
    }
    try:
        response = requests.get(API_URL, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error fetching volume issues: {e}')
        print(f'Response content: {response.content}')  # Print response content for debugging
        return {}

In [5]:
# Function to fetch detailed information for a single issue
def fetch_issue(api_detail_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY,
        'format': 'json'
    }
    try:
        response = requests.get(api_detail_url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error fetching issue details: {e}')
        return {}

In [7]:
b0 = fetch_volume_issues(98910, 0)

In [8]:
data = b0['results']

In [9]:
data1 = data.get('issues')

In [10]:
table_data = []
for issue in data1:
    url = issue.get('api_detail_url')
    details = fetch_issue(url).get('results', {})
    table_data.append(details)
    #time.sleep(20)

In [11]:
df = pd.DataFrame(table_data)

In [12]:
df.to_csv('data/super_sons.csv')

In [ ]:
def combined_issues(volume_name):
    volume_id = fetch_volume_id(volume_name)
    if not volume_id:
        print(f'Volume "{volume_name}" not found.')
        return pd.DataFrame()

    all_issues = []
    offset = 0

    try:
        while True:
            response = fetch_volume_issues(volume_id, offset)
            issues = response.get('results', [])
            all_issues.extend(issues)

            if not issues:
                break
            
            if not response.get('number_of_page_results'):
                break
            
            offset += 100  # Adjust as needed based on the API limit
            time.sleep(3)  # To avoid hitting API rate limits

    except Exception as e:
        print(f'Error occurred: {e}', offset)
        return pd.DataFrame()

    table_data = []

    for issue in all_issues:
        if isinstance(issue, dict):
            detail_url = issue.get('api_detail_url')
            if detail_url:
                details = fetch_issue(detail_url).get('results', {})
                table_data.append(details)
                time.sleep(3)  # To avoid hitting API rate limits

    df = pd.DataFrame(table_data)
    return df

# Testing Function #

In [13]:
def fetch_character():
    API_URL = BASE_URL + "characters/"
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY, 
        'format': 'json',
        'filter': 'name:Batman',
    }
    response = requests.get(API_URL, headers=headers, params=params)
    data = response.json()
    test = data["results"]
    table_data = []
    for result in test:
           # test_info = {
           #     'Name': result.get('name', 'N/A'),
           #     'Issue Credits': result.get('issue_credits', 'N/A')
           # }
            table_data.append(result)
    df = pd.DataFrame(table_data)
    return df

In [14]:
tmp = testing()
tmp

,aliases,api_detail_url,birth,count_of_issue_appearances,date_added,date_last_updated,deck,description,first_appeared_in_issue,gender,id,image,name,origin,publisher,real_name,site_detail_url
0,Bruce Wayne\nThe Caped Crusader\nThe Dark Knig...,https://comicvine.gamespot.com/api/character/4...,"Feb 19, 1972",23846,2008-06-06 11:26:57,2023-10-21 13:25:47,"Bruce Wayne, who witnessed the murder of his b...","<h2>Origin</h2><figure data-ref-id=""1300-78896...",{'api_detail_url': 'https://comicvine.gamespot...,1,1699,{'icon_url': 'https://comicvine.gamespot.com/a...,Batman,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Bruce Thomas Wayne,https://comicvine.gamespot.com/batman/4005-1699/
1,Bruce Wayne Junior\r\nBatman Jr.\r\nSuper Son\...,https://comicvine.gamespot.com/api/character/4...,None,73,2008-06-06 11:27:13,2013-06-07 19:41:36,Some Batman story lines feature a college age ...,"<h2> History</h2><p>Bruce Wayne, Jr. was the ...",{'api_detail_url': 'https://comicvine.gamespot...,1,16818,{'icon_url': 'https://comicvine.gamespot.com/a...,Batman Junior,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Bruce Wayne Junior,https://comicvine.gamespot.com/batman-junior/4...
2,None,https://comicvine.gamespot.com/api/character/4...,None,25,2008-06-06 11:25:05,2017-09-23 01:02:11,Sometimes the Caped Crusader needs a robot rep...,"<figure data-align=""left"" data-size=""medium"" d...",{'api_detail_url': 'https://comicvine.gamespot...,1,53183,{'icon_url': 'https://comicvine.gamespot.com/a...,Batman Robot,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,None,https://comicvine.gamespot.com/batman-robot/40...
3,Batman Predator,https://comicvine.gamespot.com/api/character/4...,None,2,2008-08-02 18:35:15,2023-09-20 07:23:44,Batman Predator is one of the Metapredators. ...,None,{'api_detail_url': 'https://comicvine.gamespot...,1,56707,{'icon_url': 'https://comicvine.gamespot.com/a...,Batman Predator,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,None,https://comicvine.gamespot.com/batman-predator...
4,None,https://comicvine.gamespot.com/api/character/4...,None,1,2008-12-29 22:16:13,2013-04-22 17:09:06,None,,{'api_detail_url': 'https://comicvine.gamespot...,1,59062,{'icon_url': 'https://comicvine.gamespot.com/a...,King Batman,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Bruce Wayne XXIII,https://comicvine.gamespot.com/king-batman/400...
5,Bruce Wayne,https://comicvine.gamespot.com/api/character/4...,None,49,2009-05-07 16:46:14,2022-02-05 01:12:21,The Dark Knight of Earth-22 may be old and cri...,<p>After waging a personal war on crime for de...,{'api_detail_url': 'https://comicvine.gamespot...,1,61907,{'icon_url': 'https://comicvine.gamespot.com/a...,Kingdom Come Batman,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Bruce Wayne,https://comicvine.gamespot.com/kingdom-come-ba...
6,Mr. Jones,https://comicvine.gamespot.com/api/character/4...,None,8,2009-09-18 11:10:14,2011-10-19 18:09:50,"After being rescued by Batman, Mr & Mrs Jones ...",<p>Batman Jones reappeared as a Batman and org...,{'api_detail_url': 'https://comicvine.gamespot...,1,66167,{'icon_url': 'https://comicvine.gamespot.com/a...,Batman Jones,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Batman Jones,https://comicvine.gamespot.com/batman-jones/40...
7,The Dark Knight of Zur-En-Arrh\nBatman of Z.E....,https://comicvine.gamespot.com/api/character/4...,None,37,2010-03-04 08:01:27,2024-03-29 09:54:04,This incarnation of Batman is The Dark Knight ...,<h2><strong>Silver Age</strong></h2><p>The Dar...,{'api_detail_url': 'https://comicvine.gamespot...,1,69695,{'icon_url': 'https://comicvine.gamespot.com/a...,Batman of Zur-En-Arrh,{'api_d

In [19]:
def fetch_character_details(api_detail_url):
    API_URL = api_detail_url
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY, 
        'format': 'json'
    }
    response = requests.get(API_URL, headers=headers, params=params)
    data = response.json()
    
    return data
    
    

In [25]:
batman = fetch_character_details('https://comicvine.gamespot.com/api/character/4/')

In [26]:
batman['results']

[]

In [7]:
def fetch_issues(query):
    API_URL = BASE_URL + "issues/"
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY, 
        'format': 'json',
        'filter': f'name:{query}',
    }
    response = requests.get(API_URL, headers=headers, params=params)
    data = response.json()
    return data

def fetch_issue(api_detail_url):
    API_URL = api_detail_url
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY, 
        'format': 'json'
    }
    response = requests.get(API_URL, headers=headers, params=params)
    data = response.json()
    return data

In [2]:
def combined_issues():
    results = fetch_issues()['results']
    table_data = []
    for result in results:
        tmp = fetch_issue(result.get('api_detail_url'))['results']
        info = {
            'ID': tmp.get('id', 'N/A'),
            'name': tmp.get('name', 'N/A'),
            'deck': tmp.get('deck', 'N/A'),
            'description': tmp.get('description'),
        }
        table_data.append(info)
    df = pd.DataFrame(table_data)
    return df
        
    

In [8]:
df = combined_issues()

In [9]:
df

,name,ID
0,The Lost Race,6
1,None,7
2,None,8
3,None,9
4,None,10
...,...,...
95,None,148
96,None,149
97,None,150
98,None,152


# Other Functions #

In [ ]:
def search_volume():

In [11]:
def search_issues(character_name=None):
    API_URL = BASE_URL + "search/"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'limit': 50,
        'query': character_name,
        'resources': 'issue'
    }
    response = requests.get(API_URL, headers=headers, params=params)
    if response.status_code == 200:
        search_data = response.json()
        return search_data
    else:
        print(f"Status code: {response.status_code}")
        return None


In [10]:
def print_issues_table(data):
    issues = data["results"]
    table_data = []
    for i in issues:
        i_info = {
            "ID": i.get("id", "N/A"),
            "Name": i.get("name", "N/A"),
            'Deck': i.get('deck', 'N/A'),
            "Description": i.get('description', 'N/A')
        }
        table_data.append(i_info)
    df = pd.DataFrame(table_data)
    return df

In [12]:
tmp = search_issues("Batman")
if tmp:
    table = print_issues_table(tmp)
table

,ID,Name,Deck,Description
0,1032945,None,None,None
1,1032941,None,None,None
2,1032943,None,None,None
3,1032942,None,None,None
4,1032938,None,None,None
5,1032939,None,None,None
6,1032936,None,None,None
7,994889,None,None,None
8,1009236,Vol. 2: The Bat-Man of Gotham,None,<p><strong><em>Gotham City has never been dark...
9,986896,The Bat-Man of Gotham: Conclusion,None,<p><em>It's Batman versus Red Mask in a brutal...


**Search Character**

In [6]:
def search_character(character_name):
    API_URL = BASE_URL + "search/"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'limit': 50, #number of characters
        'query': character_name,
        'resources': 'character',
        'filter': 
    }
    response = requests.get(API_URL, headers=headers, params=params)
    if response.status_code == 200:
        search_data = response.json()
        return search_data
    else:
        print(f"Status code: {response.status_code}")
        return None


In [7]:
def print_search_character_table(data):
    # Assuming the results are in a 'results' key in the JSON response
    characters = data['results']
    
    # Extract relevant information
    table_data = []
    for char in characters:
        char_info = {
            'Name': char['name'],
            'Real Name': char.get('real_name', 'N/A'),
            'Publisher': char['publisher']['name'] if 'publisher' in char and char['publisher'] else 'N/A',
            'Deck': char.get('deck', 'N/A'),
            "Count Appearances": char.get("count_of_issue_appearances", "N/A"),
            "Issue Credits": char['issue_credits'][0] if char.get('issue_credits') else 'N/A'
        }
        table_data.append(char_info)
    
    # Create DataFrame
    df = pd.DataFrame(table_data)
    return df

In [39]:
result = search_character("Batman")
if result:
    data = print_search_character_table(result)



In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               50 non-null     object
 1   Real Name          43 non-null     object
 2   Publisher          50 non-null     object
 3   Deck               49 non-null     object
 4   Count Appearances  50 non-null     int64 
 5   Issue Credits      50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


In [40]:
data

,Name,Real Name,Publisher,Deck,Count Appearances,Issue Credits
0,Batman,Bruce Thomas Wayne,DC Comics,"Bruce Wayne, who witnessed the murder of his b...",23846,N/A
1,Batman,None,Shueisha,None,2,N/A
2,Batman,None,DC Comics,Batman is a hero who looks like Dracula.,2,N/A
3,Bat-Man,Wang Baixi,DC Comics,"Wang Baixi is a tech wiz and skilled fighter, ...",53,N/A
4,Bat-Man,Sir William,DC Comics,The Bat-Man was a knight from the time of King...,25,N/A
5,Bat-Man,Bryce Gawain,DC Comics,Superhero in a distant future inspired by the...,1,N/A
6,Batman of 2050,Leonard Cohen,DC Comics,"When his parents are murdered, he emulates the...",2,N/A
7,Batman Who Laughs,Bruce Wayne,DC Comics,The Batman Who Laughs is the Dark Knight of Ea...,153,N/A
8,Dick Grayson,Richard John Grayson-Wayne,DC Comics,"As the first Robin, Dick Grayson was the most ...",9576,N/A
9,James Gordon,James Worthington Gordon,DC Comics,James Gordon is one of Batman's first supporte...,5284,N/A


In [10]:
import requests
import pandas as pd

API_KEY = "f7ca7f049ce4f06afd6c5c1e082061cd34779ff9"
BASE_URL = 'https://comicvine.gamespot.com/api/'

def search_character(character_name):
    API_URL = BASE_URL + "search/"
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'query': character_name,
        'resources': 'character'
    }
    response = requests.get(API_URL, headers=headers, params=params)
    if response.status_code == 200:
        search_data = response.json()
        return search_data
    else:
        print(f"Status code: {response.status_code}")
        return None

def print_table(data):
    # Assuming the results are in a 'results' key in the JSON response
    characters = data['results']
    
    # Extract relevant information
    table_data = []
    for char in characters:
        char_info = {
            'Name': char['name'],
            'Real Name': char.get('real_name', 'N/A'),
            'Publisher': char['publisher']['name'] if 'publisher' in char and char['publisher'] else 'N/A',
            'Deck': char.get('deck', 'N/A')  # Adding a brief description if available
        }
        table_data.append(char_info)
    
    # Create DataFrame
    df = pd.DataFrame(table_data)
    return df

result = search_character("Batman")
if result:
    tmp = print_table(result)
tmp

,Name,Real Name,Publisher,Deck
0,Batman,Bruce Thomas Wayne,DC Comics,"Bruce Wayne, who witnessed the murder of his b..."
1,Batman,None,Shueisha,None
2,Batman,None,DC Comics,Batman is a hero who looks like Dracula.
3,Bat-Man,Wang Baixi,DC Comics,"Wang Baixi is a tech wiz and skilled fighter, ..."
4,Bat-Man,Sir William,DC Comics,The Bat-Man was a knight from the time of King...
5,Bat-Man,Bryce Gawain,DC Comics,Superhero in a distant future inspired by the...
6,Batman of 2050,Leonard Cohen,DC Comics,"When his parents are murdered, he emulates the..."
7,Batman Who Laughs,Bruce Wayne,DC Comics,The Batman Who Laughs is the Dark Knight of Ea...
8,Dick Grayson,Richard John Grayson-Wayne,DC Comics,"As the first Robin, Dick Grayson was the most ..."
9,James Gordon,James Worthington Gordon,DC Comics,James Gordon is one of Batman's first supporte...
